# Denoising Autoencoder

In [1]:
import logging

from fastai.basics import *
from fastai.callback.all import *
from fastai.torch_basics import *

from IPython.display import display

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import vaep
from vaep.io import datasplits
from vaep.models import ae
import vaep.models as models
import vaep.model
from vaep.analyzers import analyzers

# overwriting Recorder callback with custom plot_loss
from vaep.models import plot_loss
from fastai import learner

learner.Recorder.plot_loss = plot_loss


logger = vaep.logging.setup_logger(logging.getLogger('vaep'))
logger.info(
    "Experiment 03 - Analysis of latent spaces and performance comparisions")

figures = {}  # collection of ax or figures

vaep - INFO     Experiment 03 - Analysis of latent spaces and performance comparisions


In [2]:
# catch passed parameters
args = None
args = dict(globals()).keys()

Papermill script parameters:

In [3]:
# files and folders
# Datasplit folder with data for experiment
folder_experiment: str = 'runs/example'
folder_data: str = ''  # specify data directory if needed
file_format: str = 'csv'  # file format of create splits, default pickle (pkl)
# Machine parsed metadata from rawfile workflow
fn_rawfile_metadata: str = 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv'
# training
epochs_max: int = 50  # Maximum number of epochs
# early_stopping:bool = True # Wheather to use early stopping or not
patience: int = 25  # Patience for early stopping
batch_size: int = 64  # Batch size for training (and evaluation)
cuda: bool = True  # Whether to use a GPU for training
# model
# Dimensionality of encoding dimension (latent space of model)
latent_dim: int = 25
# A underscore separated string of layers, '128_64' for the encoder, reverse will be use for decoder
hidden_layers: str = '512'

sample_idx_position: int = 0  # position of index which is sample ID
model: str = 'DAE'  # model name
model_key: str = 'DAE'  # potentially alternative key for model (grid search)
save_pred_real_na: bool = True  # Save all predictions for missing values
# metadata -> defaults for metadata extracted from machine data
meta_date_col: str = None  # date column in meta data
meta_cat_col: str = None  # category column in meta data

In [4]:
# Parameters
model = "DAE"
file_format = "csv"
latent_dim = 10
batch_size = 10
epochs_max = 200
hidden_layers = "512"
sample_idx_position = 0
cuda = False
save_pred_real_na = True
fn_rawfile_metadata = "data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv"
folder_experiment = "runs/dev_dataset_small/proteinGroups_N50"
model_key = "DAE"


Some argument transformations

In [5]:
args = vaep.nb.get_params(args, globals=globals())
args

{'folder_experiment': 'runs/dev_dataset_small/proteinGroups_N50',
 'folder_data': '',
 'file_format': 'csv',
 'fn_rawfile_metadata': 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv',
 'epochs_max': 200,
 'patience': 25,
 'batch_size': 10,
 'cuda': False,
 'latent_dim': 10,
 'hidden_layers': '512',
 'sample_idx_position': 0,
 'model': 'DAE',
 'model_key': 'DAE',
 'save_pred_real_na': True,
 'meta_date_col': None,
 'meta_cat_col': None}

In [6]:
args = vaep.nb.args_from_dict(args)

if isinstance(args.hidden_layers, str):
    args.overwrite_entry("hidden_layers", [int(x)
                         for x in args.hidden_layers.split('_')])
else:
    raise ValueError(
        f"hidden_layers is of unknown type {type(args.hidden_layers)}")
args

{'batch_size': 10,
 'cuda': False,
 'data': Path('runs/dev_dataset_small/proteinGroups_N50/data'),
 'epochs_max': 200,
 'file_format': 'csv',
 'fn_rawfile_metadata': 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv',
 'folder_data': '',
 'folder_experiment': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'hidden_layers': [512],
 'latent_dim': 10,
 'meta_cat_col': None,
 'meta_date_col': None,
 'model': 'DAE',
 'model_key': 'DAE',
 'out_figures': Path('runs/dev_dataset_small/proteinGroups_N50/figures'),
 'out_folder': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'out_metrics': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'out_models': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'out_preds': Path('runs/dev_dataset_small/proteinGroups_N50/preds'),
 'patience': 25,
 'sample_idx_position': 0,
 'save_pred_real_na': True}

Some naming conventions

In [7]:
TEMPLATE_MODEL_PARAMS = 'model_params_{}.json'

## Load data in long format

In [8]:
data = datasplits.DataSplits.from_folder(
    args.data, file_format=args.file_format)

vaep.io.datasplits - INFO     Loaded 'train_X' from file: runs/dev_dataset_small/proteinGroups_N50/data/train_X.csv


vaep.io.datasplits - INFO     Loaded 'val_y' from file: runs/dev_dataset_small/proteinGroups_N50/data/val_y.csv


vaep.io.datasplits - INFO     Loaded 'test_y' from file: runs/dev_dataset_small/proteinGroups_N50/data/test_y.csv


data is loaded in long format

In [9]:
data.train_X.sample(5)

Sample ID                                       Gene Names
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070  NDC80        24.575
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070  RPL5         32.980
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070  CAPNS1       27.820
                                                TUBGCP3      24.820
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070  FKBP2        27.763
Name: intensity, dtype: float64

Infer index names from long format

In [10]:
index_columns = list(data.train_X.index.names)
sample_id = index_columns.pop(args.sample_idx_position)
if len(index_columns) == 1:
    index_column = index_columns.pop()
    index_columns = None
    logger.info(f"{sample_id = }, single feature: {index_column = }")
else:
    logger.info(f"{sample_id = }, multiple features: {index_columns = }")

if not index_columns:
    index_columns = [sample_id, index_column]
else:
    raise NotImplementedError(
        "More than one feature: Needs to be implemented. see above logging output.")

vaep - INFO     sample_id = 'Sample ID', single feature: index_column = 'Gene Names'


load meta data for splits

In [11]:
if args.fn_rawfile_metadata:
    df_meta = pd.read_csv(args.fn_rawfile_metadata, index_col=0)
    display(df_meta.loc[data.train_X.index.levels[0]])
else:
    df_meta = None

Version Content Creation Date  \
Sample ID                                                                       
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-18 14:35:26   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-19 19:48:33   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-20 14:15:00   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-27 12:29:44   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-29 15:06:50   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070       66   2019-12-29 18:18:31   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-02 17:38:26   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-03 11:17:32   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-03 16:58:20   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-03 20:10:03   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-04 04:23:07   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-04 10:03:55   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-04 14:59:27   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-06 20:17:34   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-08 16:43:19   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-09 11:07:52   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-15 13:56:57   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070       66   2020-01-20 15:10:06   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-05 20:55:39   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-10 15:41:47   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-11 10:35:29   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-12 05:06:18   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-13 00:26:52   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-13 03:11:08   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-17 13:55:21   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-18 01:25:20   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-18 18:55:51   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070       66   2020-02-28 12:27:07   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-01 23:00:51   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-06 16:22:33   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-07 18:15:03   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070       66   2020-03-11 11:25:29   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-04 11:39:52   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-12 15:13:44   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-12 18:10:40   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-12 21:07:35   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-14 14:46:12   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-14 17:28:58   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-14 20:11:31   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-15 10:30:10   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-20 12:33:40   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-20 15:35:03   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-22 14:57:09   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-22 17:43:19   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-26 14:20:17   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-27 13:57:57   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070       66   2020-05-28 04:06:33   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070       66   2020-06-01 10:22:06  

### Produce some addional simulated samples

The validation simulated NA is used to by all models to evaluate training performance.

In [12]:
val_pred_simulated_na = data.val_y.to_frame(name='observed')
val_pred_simulated_na

observed
Sample ID                                      Gene Names                     
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070 SEH1L                    27.180
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070 SLC9A3R1                 30.259
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 KYNU                     30.629
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070 ADD3                     24.541
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070 CALM2;CALM1              33.934
...                                                                        ...
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 PPP2R4;DKFZp781M17165    28.717
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070 MED23                    24.966
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070 RBMXL1                   26.654
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070 SMARCA1                  27.917
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070 RGPD3;RGPD4              24.289

[9327 rows x 1 columns]

In [13]:
test_pred_simulated_na = data.test_y.to_frame(name='observed')
test_pred_simulated_na.describe()

observed
count 9,327.000
mean     27.230
std       2.888
min      19.974
25%      24.905
50%      26.929
75%      29.174
max      37.689

## Data in wide format

- Autoencoder need data in wide format

In [14]:
data.to_wide_format()
args.M = data.train_X.shape[-1]
data.train_X.head()

Gene Names                                       AAAS   AACS  AAMDC   AAMP  \
Sample ID                                                                    
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 28.349 26.133    NaN 26.777   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070 27.657 25.019 24.236 26.271   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 28.352 23.740    NaN 27.098   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070 26.826    NaN    NaN    NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 27.404 26.948 23.864 26.982   

Gene Names                                       AAR2   AARS  AARS2  AASDHPPT  \
Sample ID                                                                       
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 27.248 32.195 27.153    27.872   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 26.530    28.191   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 27.377 32.885 27.515    28.476   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070    NaN 31.926 26.157    27.635   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 26.520 31.844 25.342    27.416   

Gene Names                                       AATF  ABCB10  ...  ZNHIT2  \
Sample ID                                                      ...           
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 28.603  26.110  ...     NaN   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070 27.942  25.735  ...     NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 28.771  26.787  ...  25.184   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070 27.851  25.346  ...     NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 27.474     NaN  ...  24.591   

Gene Names                                      ZNRF2   ZPR1  ZRANB2   ZW10  \
Sample ID                                                                     
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070    NaN 28.309  28.831 28.851   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN 28.133  27.267    NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070    NaN 28.968  27.322 29.388   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070 23.586    NaN  27.278 28.076   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070    NaN 25.838  29.326 27.491   

Gene Names                                      ZWILCH  ZWINT    ZYX  \
Sample ID                                                              
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070     NaN 24.379 29.427   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070     NaN    NaN    NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070     NaN 25.803 29.525   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070     NaN 24.688 28.783   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070  26.406    NaN 30.422   

Gene Names                                      hCG_2014768;TMA7  pk;ZAK  
Sample ID                                                                 
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070            28.080     NaN  
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070               NaN     NaN  
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070               NaN     NaN  
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070               NaN     NaN  
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070               NaN  26.939  

[5 rows x 4405 columns]

### Fill Validation data with potentially missing features

In [15]:
data.train_X

Gene Names                                       AAAS   AACS  AAMDC   AAMP  \
Sample ID                                                                    
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 28.349 26.133    NaN 26.777   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070 27.657 25.019 24.236 26.271   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070 28.352 23.740    NaN 27.098   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070 26.826    NaN    NaN    NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070 27.404 26.948 23.864 26.982   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070 27.891 26.481 26.348 27.849   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070 25.498    NaN    NaN    NaN   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070 27.352    NaN    NaN 25.275   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070 27.620 25.624 23.520 27.136   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070 27.300    NaN 25.660    NaN   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070    NaN 27.612 26.211 27.912   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 23.081 25.780   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070 24.616    NaN    NaN 24.765   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070 28.265 27.735 27.055 28.733   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070    NaN 25.442    NaN 25.347   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070 26.167    NaN    NaN 26.877   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070 26.019 23.935    NaN 24.982   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070 28.735    NaN 26.648 29.133   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070 28.157 27.489 25.840 28.594   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070 29.216 27.557    NaN 27.454   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070 29.156 27.582 25.516 27.251   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070 29.255 27.394 26.058 27.348   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070 29.606 27.286 26.377 26.431   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070 27.712 26.283    NaN    NaN   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070 28.435 26.624 25.520 26.746   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070 26.691 25.331    NaN 25.831   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070 29.169    NaN 23.877 26.146   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070 27.702 25.953    NaN 26.582   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070    NaN 25.257    NaN 24.869   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 22.751 25.799   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070 26.891 24.577 25.052 26.740   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070 27.699 26.344    NaN 25.791   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070 26.127 25.320    NaN 25.801   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070 27.138 25.875    NaN 25.722   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070 26.844 24.719 25.460 25.282   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070 26.853 25.421    NaN 24.534   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070 26.203 24.863 23.161 24.822   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070 27.203 23.885 23.234 25.293   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070 27.429 25.154 23.142 25.126   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070 27.494 25.606 25.852 25.617   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070 27.396 25.552    NaN 25.741   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 27.721 24.916    NaN    NaN   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070 27.808 24.714    NaN 25.857   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070 28.051 25.608 26.030    NaN   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070 27.325 26.800 25.519 25.204   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070 29.119 27.061 25.989 27.999   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070 30.080 27.373 26.760 27.601   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070 27.298    NaN    NaN 28.318   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070 27.121    NaN    NaN 27.968   
2020_06_02_09_41_Q-Exa

In [16]:
data.val_y  # potentially has less features

Gene Names                                       AAAS   AACS  AAMDC   AAMP  \
Sample ID                                                                    
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 24.433    NaN   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN 27.733   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070 27.594    NaN    NaN    NaN   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070 22.666    NaN    NaN    NaN   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070    NaN 25.257    NaN    NaN   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 25.797    NaN   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN 26.762   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 24.649    NaN   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070    NaN 23.806    NaN    NaN   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 24.125    NaN   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 23.002    NaN   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_06_02_09_41_Q-Exa

In [17]:
data.val_y = pd.DataFrame(pd.NA, index=data.train_X.index,
                          columns=data.train_X.columns).fillna(data.val_y)
data.val_y

Gene Names                                       AAAS   AACS  AAMDC   AAMP  \
Sample ID                                                                    
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 24.433    NaN   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN 27.733   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070 27.594    NaN    NaN    NaN   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070 22.666    NaN    NaN    NaN   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070    NaN 25.257    NaN    NaN   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 25.797    NaN   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN 26.762   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 24.649    NaN   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070    NaN 23.806    NaN    NaN   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 24.125    NaN   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN 23.002    NaN   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070    NaN    NaN    NaN    NaN   
2020_06_02_09_41_Q-Exa

## Denoising Autoencoder

### Analysis: DataLoaders, Model, transform

In [18]:
default_pipeline = sklearn.pipeline.Pipeline(
    [
        ('normalize', StandardScaler()),
        ('impute', SimpleImputer(add_indicator=False))
    ])

analysis = ae.AutoEncoderAnalysis(
    train_df=data.train_X,
    val_df=data.val_y,
    model=ae.Autoencoder,
    transform=default_pipeline,
    decode=['normalize'],
    model_kwargs=dict(n_features=data.train_X.shape[-1],
                      n_neurons=args.hidden_layers,
                      last_decoder_activation=None,
                      dim_latent=args.latent_dim),
    bs=args.batch_size)
args.n_params = analysis.n_params_ae

if args.cuda:
    analysis.model = analysis.model.cuda()
analysis.model

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=4405, out_features=512, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.1)
    (4): Linear(in_features=512, out_features=4405, bias=True)
  )
)

### Training

In [19]:
analysis.learn = Learner(dls=analysis.dls,
                         model=analysis.model,
                         loss_func=MSELossFlat(reduction='sum'),
                         cbs=[EarlyStoppingCallback(patience=args.patience),
                              ae.ModelAdapter(p=0.2)]
                         )

analysis.learn.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, Recorder, ProgressCallback, EarlyStoppingCallback]
  Start Epoch Loop
     - before_epoch   : [Recorder, ProgressCallback]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - before_batch   : [ModelAdapter, CastToTensor]
         - after_pred     : [ModelAdapter]
         - after_loss     : [ModelAdapter]
         - before_backward: []
         - before_step    : []
         - after_step     : []
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder, ProgressCallback]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder, ProgressCallback]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - **CBs same as train batch**: []
      End Batch Loop
    End Valid
     - after_cancel_validate: [Recorder]
     - a

Adding a `EarlyStoppingCallback` results in an error.  Potential fix in
[PR3509](https://github.com/fastai/fastai/pull/3509) is not yet in
current version. Try again later

In [20]:
# learn.summary()

In [21]:
suggested_lr = analysis.learn.lr_find()
analysis.params['suggested_inital_lr'] = suggested_lr.valley
suggested_lr

SuggestedLRs(valley=0.0012022644514217973)

dump model config

In [22]:
vaep.io.dump_json(analysis.params, args.out_models /
                  TEMPLATE_MODEL_PARAMS.format(args.model_key))

In [23]:
# papermill_description=train
analysis.learn.fit_one_cycle(args.epochs_max, lr_max=suggested_lr.valley)

No improvement since epoch 27: early stopping


Save number of actually trained epochs

In [24]:
args.epoch_trained = analysis.learn.epoch + 1
args.epoch_trained

53

#### Loss normalized by total number of measurements

In [25]:
N_train_notna = data.train_X.notna().sum().sum()
N_val_notna = data.val_y.notna().sum().sum()
fig = models.plot_training_losses(analysis.learn, args.model_key,
                                  folder=args.out_figures,
                                  norm_factors=[N_train_notna, N_val_notna])

vaep.plotting - INFO     Saved Figures to runs/dev_dataset_small/proteinGroups_N50/figures/dae_training


Why is the validation loss better then the training loss?
- during training input data is masked and needs to be reconstructed
- when evaluating the model, all input data is provided and only the artifically masked data is used for evaluation.

### Predictions

- data of training data set and validation dataset to create predictions is the same as training data.
- predictions include missing values (which are not further compared)

- [ ] double check ModelAdapter

create predictiona and select for validation data

In [26]:
analysis.model.eval()
pred, target = analysis.get_preds_from_df(df_wide=data.train_X)  # train_X
pred = pred.stack()
pred

Sample ID                                       Gene Names      
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070  AAAS               27.718
                                                AACS               25.908
                                                AAMDC              25.217
                                                AAMP               26.631
                                                AAR2               26.560
                                                                    ...  
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070  ZWILCH             24.522
                                                ZWINT              26.541
                                                ZYX                29.920
                                                hCG_2014768;TMA7   28.729
                                                pk;ZAK             24.710
Length: 220250, dtype: float32

In [27]:
val_pred_simulated_na['DAE'] = pred  # model_key ?
val_pred_simulated_na

observed  \
Sample ID                                      Gene Names                        
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070 SEH1L                    27.180   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070 SLC9A3R1                 30.259   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 KYNU                     30.629   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070 ADD3                     24.541   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070 CALM2;CALM1              33.934   
...                                                                        ...   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 PPP2R4;DKFZp781M17165    28.717   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070 MED23                    24.966   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070 RBMXL1                   26.654   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070 SMARCA1                  27.917   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070 RGPD3;RGPD4              24.289   

                                                                        DAE  
Sample ID                                      Gene Names                    
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070 SEH1L                 27.106  
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070 SLC9A3R1              27.971  
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 KYNU                  30.852  
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070 ADD3                  25.358  
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070 CALM2;CALM1           33.375  
...                                                                     ...  
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070 PPP2R4;DKFZp781M17165 28.157  
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070 MED23                 24.061  
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070 RBMXL1                24.669  
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070 SMARCA1               27.986  
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070 RGPD3;RGPD4           24.628  

[9327 rows x 2 columns]

In [28]:
test_pred_simulated_na['DAE'] = pred  # model_key?
test_pred_simulated_na

observed  \
Sample ID                                      Gene Names                   
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 ABCE1               29.538   
                                               ACTN4               33.270   
                                               ADD3                24.259   
                                               ALCAM               24.906   
                                               ALDH1B1             26.674   
...                                                                   ...   
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070 YIPF3               25.040   
                                               YTHDF3              28.717   
                                               ZC3H11A             28.935   
                                               ZFP91-CNTF;ZFP91    25.662   
                                               hCG_2014768;TMA7    29.212   

                                                                   DAE  
Sample ID                                      Gene Names               
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070 ABCE1            29.971  
                                               ACTN4            33.536  
                                               ADD3             25.298  
                                               ALCAM            25.283  
                                               ALDH1B1          27.222  
...                                                                ...  
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070 YIPF3            26.066  
                                               YTHDF3           28.329  
                                               ZC3H11A          30.068  
                                               ZFP91-CNTF;ZFP91 27.210  
                                               hCG_2014768;TMA7 28.729  

[9327 rows x 2 columns]

save missing values predictions

In [29]:
if args.save_pred_real_na:
    pred_real_na = ae.get_missing_values(df_train_wide=data.train_X,
                                         val_idx=val_pred_simulated_na.index,
                                         test_idx=test_pred_simulated_na.index,
                                         pred=pred)
    display(pred_real_na)
    pred_real_na.to_csv(args.out_preds / f"pred_real_na_{args.model_key}.csv")

Sample ID                                       Gene Names 
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070  AAMDC         25.217
                                                ABCB7         24.814
                                                ABCC4         24.133
                                                ABI2          24.911
                                                ACOT2;ACOT1   27.099
                                                               ...  
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070  ZNF346        24.881
                                                ZNF706        25.352
                                                ZNRF2         25.235
                                                ZWILCH        24.522
                                                pk;ZAK        24.710
Name: intensity, Length: 33698, dtype: float32

### Plots

- validation data

In [30]:
analysis.model.cpu()
df_latent = vaep.model.get_latent_space(analysis.model.encoder,
                                        dl=analysis.dls.valid,
                                        dl_index=analysis.dls.valid.data.index)
df_latent

latent dimension 1  \
Sample ID                                                            
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070               0.250   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070              -1.108   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070               0.378   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070              -0.869   
2019_12_29_15_06_Q-Exactive-HF-X-Orbitrap_6070               1.172   
2019_12_29_18_18_Q-Exactive-HF-X-Orbitrap_6070               2.020   
2020_01_02_17_38_Q-Exactive-HF-X-Orbitrap_6070              -3.590   
2020_01_03_11_17_Q-Exactive-HF-X-Orbitrap_6070              -1.141   
2020_01_03_16_58_Q-Exactive-HF-X-Orbitrap_6070              -0.859   
2020_01_03_20_10_Q-Exactive-HF-X-Orbitrap_6070              -1.108   
2020_01_04_04_23_Q-Exactive-HF-X-Orbitrap_6070               0.891   
2020_01_04_10_03_Q-Exactive-HF-X-Orbitrap_6070              -8.432   
2020_01_04_14_59_Q-Exactive-HF-X-Orbitrap_6070              -3.912   
2020_01_06_20_17_Q-Exactive-HF-X-Orbitrap_6070               2.333   
2020_01_08_16_43_Q-Exactive-HF-X-Orbitrap_6070              -1.181   
2020_01_09_11_07_Q-Exactive-HF-X-Orbitrap_6070              -0.681   
2020_01_15_13_56_Q-Exactive-HF-X-Orbitrap_6070              -2.912   
2020_01_20_15_10_Q-Exactive-HF-X-Orbitrap_6070               3.932   
2020_02_05_20_55_Q-Exactive-HF-X-Orbitrap_6070               2.883   
2020_02_10_15_41_Q-Exactive-HF-X-Orbitrap_6070               3.315   
2020_02_11_10_35_Q-Exactive-HF-X-Orbitrap_6070               2.780   
2020_02_12_05_06_Q-Exactive-HF-X-Orbitrap_6070               3.410   
2020_02_13_00_26_Q-Exactive-HF-X-Orbitrap_6070               3.189   
2020_02_13_03_11_Q-Exactive-HF-X-Orbitrap_6070               3.563   
2020_02_17_13_55_Q-Exactive-HF-X-Orbitrap_6070               1.111   
2020_02_18_01_25_Q-Exactive-HF-X-Orbitrap_6070              -1.000   
2020_02_18_18_55_Q-Exactive-HF-X-Orbitrap_6070               1.634   
2020_02_28_12_27_Q-Exactive-HF-X-Orbitrap_6070               0.372   
2020_03_01_23_00_Q-Exactive-HF-X-Orbitrap_6070              -1.574   
2020_03_06_16_22_Q-Exactive-HF-X-Orbitrap_6070              -4.048   
2020_03_07_18_15_Q-Exactive-HF-X-Orbitrap_6070              -1.057   
2020_03_11_11_25_Q-Exactive-HF-X-Orbitrap_6070               0.553   
2020_05_04_11_39_Q-Exactive-HF-X-Orbitrap_6070              -2.028   
2020_05_12_15_13_Q-Exactive-HF-X-Orbitrap_6070              -1.975   
2020_05_12_18_10_Q-Exactive-HF-X-Orbitrap_6070              -2.801   
2020_05_12_21_07_Q-Exactive-HF-X-Orbitrap_6070              -2.841   
2020_05_14_14_46_Q-Exactive-HF-X-Orbitrap_6070              -2.201   
2020_05_14_17_28_Q-Exactive-HF-X-Orbitrap_6070              -2.579   
2020_05_14_20_11_Q-Exactive-HF-X-Orbitrap_6070              -1.802   
2020_05_15_10_30_Q-Exactive-HF-X-Orbitrap_6070              -0.978   
2020_05_20_12_33_Q-Exactive-HF-X-Orbitrap_6070              -2.347   
2020_05_20_15_35_Q-Exactive-HF-X-Orbitrap_6070              -1.545   
2020_05_22_14_57_Q-Exactive-HF-X-Orbitrap_6070              -1.375   
2020_05_22_17_43_Q-Exactive-HF-X-Orbitrap_6070              -1.304   
2020_05_26_14_20_Q-Exactive-HF-X-Orbitrap_6070               0.735   
2020_05_27_13_57_Q-Exactive-HF-X-Orbitrap_6070               4.252   
2020_05_28_04_06_Q-Exactive-HF-X-Orbitrap_6070               5.335   
2020_06_01_10_22_Q-Exactive-HF-X-Orbitrap_6070              -1.081   
2020_06_01_15_41_Q-Exactive-HF-X-Orbitrap_6070               0.903   
2020_06_02_09_41_Q-Exactive-HF-X-Orbitrap_6070               1.054   

                                                latent dimension 2  \
Sample ID                                                            
2019_12_18_14_35_Q-Exactive-HF-X-Orbitrap_6070               1.199   
2019_12_19_19_48_Q-Exactive-HF-X-Orbitrap_6070               1.792   
2019_12_20_14_15_Q-Exactive-HF-X-Orbitrap_6070               1.401   
2019_12_27_12_29_Q-Exactive-HF-X-Orbitrap_6070           

In [31]:
# ! calculate embeddings only if meta data is available? Optional argument to save embeddings?
ana_latent = analyzers.LatentAnalysis(df_latent,
                                      df_meta,
                                      args.model_key,
                                      folder=args.out_figures)
if args.meta_date_col and df_meta is not None:
    figures[f'latent_{args.model_key}_by_date'], ax = ana_latent.plot_by_date(
        args.meta_date_col)

In [32]:
if args.meta_cat_col and df_meta is not None:
    figures[f'latent_{args.model_key}_by_{"_".join(args.meta_cat_col.split())}'], ax = ana_latent.plot_by_category(
        args.meta_cat_col)

## Comparisons

Simulated NAs : Artificially created NAs. Some data was sampled and set
explicitly to misssing before it was fed to the model for
reconstruction.

### Validation data

- all measured (identified, observed) peptides in validation data

In [33]:
# papermill_description=metrics
d_metrics = models.Metrics()

The simulated NA for the validation step are real test data (not used for training nor early stopping)

In [34]:
added_metrics = d_metrics.add_metrics(val_pred_simulated_na, 'valid_simulated_na')
added_metrics

Selected as truth to compare to: observed


{'DAE': {'MSE': 0.8326519617296297,
  'MAE': 0.6277039353944084,
  'N': 9327,
  'prop': 1.0}}

### Test Datasplit

In [35]:
added_metrics = d_metrics.add_metrics(test_pred_simulated_na, 'test_simulated_na')
added_metrics

Selected as truth to compare to: observed


{'DAE': {'MSE': 0.8591098492751501,
  'MAE': 0.6295604511255216,
  'N': 9327,
  'prop': 1.0}}

Save all metrics as json

In [36]:
vaep.io.dump_json(d_metrics.metrics, args.out_metrics /
                  f'metrics_{args.model_key}.json')
d_metrics

{ 'test_simulated_na': { 'DAE': { 'MAE': 0.6295604511255216,
                                  'MSE': 0.8591098492751501,
                                  'N': 9327,
                                  'prop': 1.0}},
  'valid_simulated_na': { 'DAE': { 'MAE': 0.6277039353944084,
                                   'MSE': 0.8326519617296297,
                                   'N': 9327,
                                   'prop': 1.0}}}

In [37]:
metrics_df = models.get_df_from_nested_dict(d_metrics.metrics,
                                            column_levels=['model', 'metric_name']).T
metrics_df

subset             valid_simulated_na  test_simulated_na
model metric_name                                       
DAE   MSE                       0.833              0.859
      MAE                       0.628              0.630
      N                     9,327.000          9,327.000
      prop                      1.000              1.000

## Save predictions

In [38]:
# save simulated missing values for both splits
val_pred_simulated_na.to_csv(args.out_preds / f"pred_val_{args.model_key}.csv")
test_pred_simulated_na.to_csv(args.out_preds / f"pred_test_{args.model_key}.csv")

## Config

In [39]:
figures  # switch to fnames?

{}

In [40]:
args.dump(fname=args.out_models / f"model_config_{args.model_key}.yaml")
args

{'M': 4405,
 'batch_size': 10,
 'cuda': False,
 'data': Path('runs/dev_dataset_small/proteinGroups_N50/data'),
 'epoch_trained': 53,
 'epochs_max': 200,
 'file_format': 'csv',
 'fn_rawfile_metadata': 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv',
 'folder_data': '',
 'folder_experiment': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'hidden_layers': [512],
 'latent_dim': 10,
 'meta_cat_col': None,
 'meta_date_col': None,
 'model': 'DAE',
 'model_key': 'DAE',
 'n_params': 4528447,
 'out_figures': Path('runs/dev_dataset_small/proteinGroups_N50/figures'),
 'out_folder': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'out_metrics': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'out_models': Path('runs/dev_dataset_small/proteinGroups_N50'),
 'out_preds': Path('runs/dev_dataset_small/proteinGroups_N50/preds'),
 'patience': 25,
 'sample_idx_position': 0,
 'save_pred_real_na': True}